In [1]:
import os

from transformers import AutoModel, AutoTokenizer, pipeline

In [2]:
try:
    from google.colab import drive

    print(
        "You work on Colab. Gentle as we are, we will mount Drive for you. "
        "It'd help if you allowed this in the popup that opens."
    )
    drive.mount('/content/drive')
    DATA_DIR = os.path.join('drive', 'MyDrive', 'KEDiff', 'data')
except:
    print("You do not work on Colab")
    DATA_DIR = os.path.join('data')
    pass

print(f"{DATA_DIR=}", '-->', os.path.abspath(DATA_DIR))

You do not work on Colab
DATA_DIR='data' --> /Users/lelvilamp/source/repos/lelvilamp/kediff-ner-training/data


In [3]:
TOKENISER_CHECKPOINT = "dbmdz/bert-base-historic-multilingual-cased"
CHECKPOINT_NAME_BASE = "oalz-1788-q1-ner-"
TRAINED_DIR = os.path.join(DATA_DIR, 'trained_data')

In [4]:
tokeniser = AutoTokenizer.from_pretrained(TOKENISER_CHECKPOINT)

In [5]:
sample = {
    "id": 1003,
    "text": "Unter die Gegenden, die den Borkenk\u00e4fer n\u00e4hren S. 13 \u2014 14. geh\u00f6rt leider auch das Erzstift Salzburg.",
    "label": [[82, 99, "ORG"], [82, 99, "LOC"], [28, 39, "MISC"]]
}

In [6]:
x = tokeniser(sample['text'])

In [7]:
label_types = ["EVENT", "LOC", "MISC", "ORG", "PER", "TIME"]
ner_model_paths = {
    label_type: os.path.join(TRAINED_DIR,
                             "".join([CHECKPOINT_NAME_BASE, label_type]))
    for label_type in label_types
}

In [8]:
token_classifiers = {
    label_type : pipeline("token-classification",
                          model=ner_model_paths[label_type],
                          aggregation_strategy="simple")
    for label_type in label_types
}

In [9]:
token_classifier = token_classifiers['PER']

In [15]:
token_classifier("(Das hei\u00dft ab ovo anfangen, wie's jener that, der vom deutschen Gleichgewichte handeln wollte, und von Adam anfieng.)")

[{'entity_group': 'PER',
  'score': 0.9940697,
  'word': 'Adam',
  'start': 103,
  'end': 107}]

In [25]:
token_classifier("Daniel Göller ist der beste Masterstudent Christian Borgelts und sollte eine Ehrenmedaille… medallje… medallie… wie schreibt man das????… sowie eine saftige Sonderzahlung von Herrn Prof. Lehnert, Rektor der Universität zu Salzburg, erhalten, sodass er endlich nach Island reisen und dort in einer Kirche zu Gott beten kann.")

[{'entity_group': 'PER',
  'score': 0.99926645,
  'word': 'Daniel Göller',
  'start': 0,
  'end': 13},
 {'entity_group': 'PER',
  'score': 0.990286,
  'word': 'Christian Borgelts',
  'start': 42,
  'end': 60},
 {'entity_group': 'PER',
  'score': 0.8944866,
  'word': 'Herrn Prof. Lehnert, Rektor der Universität',
  'start': 175,
  'end': 218},
 {'entity_group': 'PER',
  'score': 0.89517885,
  'word': 'Gott',
  'start': 307,
  'end': 311}]

In [29]:
token_classifier("Nun ist die Frage, ob das Modell auch mit Frauennamen umgehen kann, da beim Lesen der Originaltexte ein deutlicher Bias zu Männernamen aufgefallen ist. Und wie es dann wohl mit geschlechtsneutralen Namen aussieht?")

[]

In [21]:
token_classifier("Bundeskanzlerin Brigitte Bierlein führte bis zur Angelobung der Bundesregierung Kurz II nach der vorgezogenen Nationalratswahl im Herbst 2019 die Amtsgeschäfte der Bundesministerien weiter. Vielleicht kennt sie ja auch Angela Merkel?")

[{'entity_group': 'PER',
  'score': 0.9992562,
  'word': 'Bundeskanzlerin Brigitte Bierlein',
  'start': 0,
  'end': 33},
 {'entity_group': 'PER',
  'score': 0.998597,
  'word': 'Angela Merkel',
  'start': 219,
  'end': 232}]

In [ ]:
token_classifier("Test in Salzburg während der Österreichsichen Aufklärung. In Paris wurden mehrere Menschen aus Deutschland gesichtet.")

In [ ]:
token_classifier("den meisten Lesern durch eine ausführliche Beschreibung und Beurtheilung des Wirtembergischen, im katholischen Deutschlande noch immer nicht genug belannten Gesangbuches, einen Gefallen zu erzeigen.")